In [ ]:
try:
  import openai
except:
  !pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
oai_key="<YOUR_OPENAI_API_KEY>"

In [ ]:
def generate_response(prompt,artefact=None,mask=None,model_engine = 'gpt-4',max_tokens=1024,secret_key=None,
                      mode='create',test=0,persona="You are a helpful assistant.",number=1,size='1024x1024'):

    # https://github.com/openai/openai-python
    # https://platform.openai.com/docs/models/overview
    # https://platform.openai.com/docs/api-reference/introduction
    # https://platform.openai.com/docs/api-reference/completions/create

    # MODELS: 'gpt-4','gpt-3.5-turbo', "text-davinci-003","text-davinci-002", "code-davinci-002", "dall.e"
    # MODE: only for image generation/editing: https://platform.openai.com/docs/guides/images/introduction - 'create','edit','variation'

    # If input is a text file
    if prompt.split('.')[-1].lower() in ['txt','dat','sql','py','r']:
        print('Reading ',prompt)
        with open(prompt) as f:
            prompt = f.read()

    # If additional data provided
    if artefact is not None:
        print('Additional artefact provided...')
        if artefact.split('.')[-1].lower() in ['txt','dat','sql','py','r']:
            print('Reading ',artefact)
            with open(artefact) as f:
                artefact = f.read()
            prompt=prompt+' '+artefact
        elif artefact.split('.')[-1].lower() in ['csv']:
            artefact = pd.read_csv(artefact)
            artefact = ', '.join(list(artefact.iloc[:,0].astype(str)))
            prompt=prompt+' '+artefact
        elif artefact.split('.')[-1].lower() in ['png','jpg','jpeg']:
            artefact=open(artefact, "rb")
        elif isinstance(artefact,str):
            prompt=prompt+' '+artefact

    if mask is not None:
      if mask.split('.')[-1].lower() in ['png','jpg','jpeg']:
        mask=open(mask, "rb")

    result = 'INVALID OPENAI KEY!!!'
    completions = None

    if secret_key is not None:

        print('>>> Querying chatGPT ',model_engine)
        #print(prompt)

        #prompt = (f"{prompt}")

        client = OpenAI(
          # defaults to os.environ.get("OPENAI_API_KEY")
          api_key=secret_key,
        )

        try:
            if np.sum(test)==0:
                if ('dall-e' in model_engine.lower())&(mode.lower()=='create'):

                    completions = client.images.generate(
                                  model=model_engine.lower(),
                                  prompt=prompt,
                                  n=number,
                                  size=size
                                )
                    result = completions['data'][0]['url']
                elif ('dall-e' in model_engine.lower())&(mode.lower()=='edit'):
                    completions = client.images.edit(
                                  image=artefact,
                                  mask=mask,
                                  prompt=prompt,
                                  n=number,
                                  size=size
                                )
                    result = completions['data'][0]['url']
                elif ('dall-e' in model_engine.lower())&(mode == 'variation'):
                    completions = client.images.create_variation(
                                image=artefact,
                                n=number,
                                size=size
                              )
                    result = completions['data'][0]['url']
                else:
                    completions = client.chat.completions.create(
                              model=model_engine.lower(),
                              messages=[{"role": "system", "content": persona},
                                        {"role": "user", "content": prompt}],
                              max_tokens=max_tokens,
                              n=number,
                              stop=None,
                              temperature=0.5
                          )
                    result = completions.choices[0].message.content
                    result = result.strip()

                print('>>> Query completed successfully :)')
                print('Token usage: ')
                print(completions.usage)
            else:
                print("API not queried - only testing...")
        except Exception as e:
            print(e)
            exc_type, exc_value, exc_traceback = sys.exc_info()
            traceback.print_exception(exc_type, exc_value, exc_traceback,limit=5, file=sys.stdout)
            result=e
    else:
        print('INVALID OPENAI KEY!!!')

    return(result,completions)

In [ ]:
prompt='How many moons does Jupyter have?'
answer,compl=generate_response(prompt,model_engine = "text-davinci-003",max_tokens=1024,secret_key=oai_key,test=0)
print(answer)
print(compl)

How many moons does Jupyter have?
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nJupyter does not have any moons."
    }
  ],
  "created": 1679094232,
  "id": "cmpl-6vDOyOAuENyIG44YuJma05RovXHxq",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 12,
    "prompt_tokens": 10,
    "total_tokens": 22
  }
}


In [ ]:
prompt="Please explain what the following function does:"

artefact='''
def solution(s):
    for i in range(len(s)):
        t = s[:i] + s[i+1:]
        if t == t[::-1]: return True

    return s == s[::-1]'''

answer,compl=generate_response(prompt,artefact=artefact,model_engine = "gpt-3.5-turbo",max_tokens=200,secret_key=oai_key,test=0)
print(answer)
print('**********************')
print(compl)

Additional artefact provided...
>>> Querying chatGPT  gpt-3.5-turbo  model with the following prompt:
Please explain what the following function do: 
def solution(s):
    for i in range(len(s)):
        t = s[:i] + s[i+1:]
        if t == t[::-1]: return True

    return s == s[::-1]
>>> Query completed successfully :)
Token usage: 
{
  "completion_tokens": 200,
  "prompt_tokens": 71,
  "total_tokens": 271
}
This function is a solution to check if a given string `s` is a palindrome or not. 

It works as follows:
- It loops through each character in the string `s` using the `range` function and the `len` function to get the length of the string.
- For each character, it creates a new string `t` by slicing the original string `s` from the beginning up to the current character (excluding it) and concatenating it with the rest of the string (excluding the current character).
- It then checks if the new string `t` is equal to its reverse using the `[::-1]` slicing operation. 
- If the new s

In [ ]:
prompt='''Given an array of numbers,
          write a python function to move all zeroes in the array to the end of it
          while maintaining the relative order of the non-zero elements.'''

answer,compl=generate_response(prompt,model_engine = "gpt-3.5-turbo",max_tokens=500,secret_key=oai_key,test=0)
print(answer)
print(compl)

>>> Querying chatGPT  gpt-3.5-turbo  model with the following prompt:
Given an array of numbers, write a python function to move all zeroes in the array to the end of it while maintaining the relative order of the non-zero elements.
>>> Query completed successfully :)
Token usage: 
{
  "completion_tokens": 190,
  "prompt_tokens": 51,
  "total_tokens": 241
}
Here is a Python function that moves all zeroes in the array to the end while maintaining the order of the non-zero elements:

```python
def move_zeroes(arr):
    # initialize a counter for non-zero elements
    count = 0
    
    # iterate through the array and move non-zero elements to the front
    for i in range(len(arr)):
        if arr[i] != 0:
            arr[count] = arr[i]
            count += 1
    
    # fill the remaining elements with zeroes
    while count < len(arr):
        arr[count] = 0
        count += 1
    
    return arr
```

Here's an example of how to use the function:

```python
arr = [0, 1, 0, 3, 12]
print(

In [ ]:
# Test the function
def move_zeroes(arr):
    # initialize a counter for non-zero elements
    count = 0

    # iterate through the array and move non-zero elements to the front
    for i in range(len(arr)):
        if arr[i] != 0:
            arr[count] = arr[i]
            count += 1

    # fill the remaining elements with zeroes
    while count < len(arr):
        arr[count] = 0
        count += 1

    return arr

move_zeroes([1,0,2,0,3,0,4])

[1, 2, 3, 4, 0, 0, 0]

In [ ]:
prompt="Write an Oracle SQL query that pulls all columns from table TABLE, filters only rows with value DATE greater than 28th Jun 2013 and then returns only rows 200 to 300 of the filtered data set"

answer,compl=generate_response(prompt,model_engine = "gpt-3.5-turbo",max_tokens=500,secret_key=oai_key,test=0)
print(answer)

>>> Querying chatGPT  gpt-3.5-turbo  model with the following prompt:
Write an Oracle SQL query that pulls all columns from table TABLE, filters only rows with value DATE greater than 28th Jun 2013 and then returns only rows 200 to 300 of the filtered data set
>>> Query completed successfully :)
Token usage: 
{
  "completion_tokens": 118,
  "prompt_tokens": 61,
  "total_tokens": 179
}
Here's the SQL query you can use:

```
SELECT *
FROM (
  SELECT ROW_NUMBER() OVER (ORDER BY DATE) AS row_num, TABLE.*
  FROM TABLE
  WHERE DATE > '2013-06-28'
)
WHERE row_num BETWEEN 200 AND 300;
```

This query uses a subquery to filter the data based on the DATE column, and then uses the ROW_NUMBER() function to assign a row number to each row in the filtered data set. Finally, the outer query selects only rows with row numbers between 200 and 300.


In [ ]:
prompt='''Write an Oracle SQL query that applies regexp_like function
          to columns COL1, COL2, COL5 and COL8 of table TABLE
          to search for pattern 'VFC1|VFP3|VFC4'
          and returns all rows of the table that return at least one match'''

answer,compl=generate_response(prompt,model_engine = "gpt-3.5-turbo",max_tokens=500,secret_key=oai_key,test=0)
print(answer)

>>> Querying chatGPT  gpt-3.5-turbo  model with the following prompt:
Write an Oracle SQL query that applies regexp_like function to columns COL1, COL2, COL5 and COL8 of table TABLE to search for pattern 'VFC1|VFP3|VFC4' and returns all rows of the table that return at least one match
>>> Query completed successfully :)
Token usage: 
{
  "completion_tokens": 154,
  "prompt_tokens": 74,
  "total_tokens": 228
}
Here is the SQL query that applies regexp_like function to columns COL1, COL2, COL5 and COL8 of table TABLE to search for pattern 'VFC1|VFP3|VFC4' and returns all rows of the table that return at least one match:

```
SELECT *
FROM TABLE
WHERE REGEXP_LIKE(COL1 || COL2 || COL5 || COL8, 'VFC1|VFP3|VFC4');
```

This query concatenates the values of columns COL1, COL2, COL5 and COL8 into one string and then applies the REGEXP_LIKE function to search for the specified pattern. The WHERE clause filters the results to only include rows where at least one match was found.


In [ ]:
prompt='''I want you to act as a programmer in Python. Let df be a very large dataframe with more than 1 million rows.
          Please simplify this code while ensuring that it is more efficient and explain why.'''

artefact='''
        for i in len(df):
            if df['column1']>8:
                df.loc[i,'column2']=8
            else:
                df.loc[i,'column2']=-20

        '''

answer,compl=generate_response(prompt,artefact=artefact,model_engine = "gpt-3.5-turbo",max_tokens=200,secret_key=oai_key,test=0)
print(answer)
print('**********************')
print(compl)

Additional artefact provided...
>>> Querying chatGPT  gpt-3.5-turbo  model with the following prompt:
I want you to act as a programmer in Python. Let df be a very large dataframe with more than 1 million rows.
          Please simplify this code while ensuring that it is more efficient and explain why. 
        for i in len(df):
            if df['column1']>8:
                df.loc[i,'column2']=8
            else:
                df.loc[i,'column2']=-20

        
>>> Query completed successfully :)
Token usage: 
{
  "completion_tokens": 167,
  "prompt_tokens": 103,
  "total_tokens": 270
}
Here's a simplified and more efficient version of the code you provided:

```
df.loc[df['column1'] > 8, 'column2'] = 8
df.loc[df['column1'] <= 8, 'column2'] = -20
```

This code uses boolean indexing to locate the rows where `column1` is greater than 8 and sets the corresponding values in `column2` to 8. Similarly, it sets the values in `column2` to -20 for the rows where `column1` is less than or e

In [ ]:
prompt='''I want you to act as a programmer in R.  Please translate this code to Python and explain what it does'''

artefact='''
pow <- function(x, y) {
result <- x^y
print(paste("Result: ", result))
}'''

answer,compl=generate_response(prompt,artefact=artefact,model_engine = "gpt-3.5-turbo",max_tokens=200,secret_key=oai_key,test=0)
print(answer)
print('**********************')
print(compl)

Additional artefact provided...
>>> Querying chatGPT  gpt-3.5-turbo  model with the following prompt:
I want you to act as a programmer in R.  Please translate this code to Python and explain what it does 
pow <- function(x, y) {
result <- x^y
print(paste("Result: ", result))
}
>>> Query completed successfully :)
Token usage: 
{
  "completion_tokens": 138,
  "prompt_tokens": 67,
  "total_tokens": 205
}
Certainly! Here is the equivalent code in Python:

```python
def pow(x, y):
    result = x ** y
    print("Result: ", result)
```

This code defines a function called `pow` that takes two arguments, `x` and `y`. Within the function, the result of raising `x` to the power of `y` is calculated using the `**` operator and stored in a variable called `result`. The function then prints a message to the console using the `print` function, which includes the result of the calculation.

Overall, this function provides a simple way to calculate and display the result of raising a number to a powe

In [ ]:
def pow(x, y):
    result = x ** y
    print("Result: ", result)

pow(2,3)

Result:  8


In [ ]:
prompt='''I want you to act as a programmer in Python.
          Please correct the errors in the code and explain how these errors would affect the code'''

artefact='''
        for i in df.shape:
            if df['column1']=8:
                    df.loc[i,'column2']=8
            else df['column1']>8:
                df.loc[j,'column2']==-20
            elseif:
                df.iloc[i,'column2']=1e4

        '''

answer,compl=generate_response(prompt,artefact=artefact,model_engine = "gpt-3.5-turbo",max_tokens=200,secret_key=oai_key,test=0)
print(answer)
print('**********************')
print(compl)

Additional artefact provided...
>>> Querying chatGPT  gpt-3.5-turbo  model with the following prompt:
I want you to act as a programmer in Python. 
          Please correct the errors in the code and explain how these errors would affect the code 
        for i in df.shape:
            if df['column1']=8:
                    df.loc[i,'column2']=8
            else df['column1']>8:
                df.loc[j,'column2']==-20
            elseif:
                df.iloc[i,'column2']=1e4

        
>>> Query completed successfully :)
Token usage: 
{
  "completion_tokens": 200,
  "prompt_tokens": 111,
  "total_tokens": 311
}
Here is the corrected code with explanations of the errors:

```python
for i in range(df.shape[0]): # use range() to iterate over the rows of the DataFrame
    if df.loc[i, 'column1'] == 8: # use == for comparison and specify the row and column of the DataFrame
        df.loc[i, 'column2'] = 8
    elif df.loc[i, 'column1'] > 8: # use elif instead of else if and specify the r